# how to generate files

### - Download markdown of the person you want to add from Roam and add it to covert_files folder

### - run this and fix file if needed

In [1]:
import re
def clean(text):
  text = " ".join(text.split())
  text = text.replace("[[", "")
  text = text.replace("]]", "")
  text = text.replace("\n", "")
  text = text.replace("*", "")
  return text

def clean_dict(dictionary):
  d = {}
  for k, v in dictionary.items():
    if v != {}:
      d[k] = {}
    if  isinstance(v, list):
        d[k] = v
    else:
      for k2, v2 in v.items():
        if v2:
            d[k][k2] = v2
  return d

In [2]:
def read_lines(filename):
    with open(f"convert_files/{filename}", 'r') as f:
        lines = f.readlines()
    lines = [clean(line) for line in lines if clean(line)!='']
    return lines

In [3]:
def extract_person_dict(lines):
  person = {}
  section="other"
  for i, line in enumerate(lines):
      if "###" in line:
          section = line.replace("- ### ","")
          if section == "Vida" or section == "Documentos":
            person[section] = []
          else:
            person[section] = {}
          pass
      elif section == "Vida":
        person[section].append(clean(line).replace("- ", ""))
      elif section == "Documentos":
        person[section].append(line)
      elif ":" in line:
        vals = line.split(":")
        # print(len(vals))
        if ("Morte" == vals[0] and (clean(vals[1]) == 'DATA, LOCAL')) or ("Sepultamento" == vals[0] and (clean(vals[1]) == 'Cemitério de')):
          person[section][vals[0]] = ''
        elif "Filhos" in vals[0]:
          filhos = []
          for filho in lines[i+1:]:
            if filho[0]=="-":
              filhos.append(filho.replace("- ", ""))
            else:
              break
          person[section]["Filhos"] = filhos
        else:
          person[section][vals[0]] = clean(":".join(vals[1:]))

  person = clean_dict(person)
  # display(person)
  return person

In [4]:
from datetime import date
def create_header(name, dates):
  timest = dates[0] if len(dates)>0 else "????"
  timeend = dates[1] if len(dates)>1 else "????"
  today = date.today().strftime("%d/%m/%Y")
  s = f"---\ntitle: \'{name} ({timest}-{timeend})\'\ndata: {today}\nnome: {name}\nlayout: pessoa\n---\n\n"
  return s

def get_personal_data(personal_data):
  lines = []
  for k, v in personal_data.items():
    if k =="Pais":
      text = v.split(" e ")
      line = f'**{k}:** [{text[0]}](){{:class="underconstruction"}} e [{text[1]}](){{:class="underconstruction"}}<br/>'
      lines.append(line)
    elif k =="Casamento":
      text = v.split(",")
      line = f'**{k}:** [{text[0]}](){{:class="underconstruction"}}, {",".join(text[1:])}<br/>'
      lines.append(line)
    elif k =="Filhos":
      line = f"**{k}:**<br/>"
      lines.append(line)
      # print(v)
      for i, filho in enumerate(v):
        line = f'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{i+1}. [{filho}](){{:class="underconstruction"}}<br/>'
        lines.append(line)
    else:
      line = f"**{k}:** {v}<br/>"
      lines.append(line)
  return "\n".join(lines)

def get_life(vida):
  nl = '\n' 
  paragraphs = "\n\n".join(vida)
  text = f"{nl}{nl}## Vida:{nl}{nl}{paragraphs}{nl}"
  return text

def get_refs(refs):
  print(refs)
  nl = '\n' 
  paragraphs = "\n\n".join(refs)

  text = f"{nl}{nl}## Referências:{nl}{nl}{paragraphs}{nl}"
  return text

In [5]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir("convert_files") if (isfile(join("convert_files", f)) and f !='.DS_Store')]
onlyfiles

['Raymundo Benno Scherer (1913-1979).md']

In [6]:
def get_time_name(filename):
  filename = filename.replace(".md", '')
  times = re.findall(r'\d+', filename)
  name = " ".join(re.sub(r'\([^)]*\)', '', filename).split())
  return times, name


In [7]:
def file_name_set(name, time):
  name_aux = "".join(name.split())
  date_aux = time[0] if len(time)>0 else ""
  return f"{name_aux}{date_aux}.md"

def save_file(name, time, person):
  f = open(f"converted/{file_name_set(name, time)}", "w")
  f.write(create_header(name, time))
  f.write(get_personal_data(person["Dados Pessoais"]))
  f.write(get_life(person["Vida"]))
  f.write(get_refs(person["Documentos"]))
  f.close()

In [8]:
for filename in onlyfiles:
  times, name = get_time_name(filename)
  lines = read_lines(filename)
  person = extract_person_dict(lines)
  save_file(name, times, person)


['[1] Certidão de Nascimento:  https://www.familysearch.org/ark:/61903/3:1:3QS7-89L5-39MX-T?cat=58014 Lajeado Livro A-28, Folha 43, Num. 224', 'Registro de Batismo: ', '[2] Certidão de Casamento: https://www.familysearch.org/ark:/61903/3:1:3QS7-L9GW-S6XW?lang=pt&i=2835 B-5 flh 74, num 2 - Marques de Souza', '[3] Certidão de Óbito:  https://www.familysearch.org/ark:/61903/3:1:3QS7-89GW-3QL6?lang=pt&i=1758 C-1 flh 147, num 191 - Marques de Souza', 'Outros:', '- [4] registro de obito do pai Frederico Jacob Theobaldo Scherer (1885-1948)', '- [5] registro de obito da mae Idalina Renaldina Scherer (1890-1973)', '- [6] obito do filho Létio Eleceu Scherer (1952-1977)', 'Agradecimento:']


In [9]:
# FOLHA = "6"
# NUM = "9"
# LINK = "https://drive.google.com/file/d/11alIGA7qSWr9gIa5GBm4zfa3JhQ1NJi1/view?usp=sharing"
# DATA = "11 janeiro 2025"
# print(F"Registro de Casamento da Paróquia Evangélica de Conventos, Livro 1914-1935 Folha {FOLHA} Num. {NUM}. Acervo Pessoal. Disponível em: [{LINK}]({LINK}). Acesso em: {DATA}.")

Registro de Casamento da Paróquia Evangélica de Conventos, Livro 1914-1935 Folha 6 Num. 9. Acervo Pessoal. Disponível em: [https://drive.google.com/file/d/11alIGA7qSWr9gIa5GBm4zfa3JhQ1NJi1/view?usp=sharing](https://drive.google.com/file/d/11alIGA7qSWr9gIa5GBm4zfa3JhQ1NJi1/view?usp=sharing). Acesso em: 11 janeiro 2025.
